In [1]:
import numpy as np
import matplotlib.pyplot as plt
from outreading import read_F_D_edges, read_many_profiles
from utils import init_rate_matrix
from analyzeprofiles import extend_profile_water, extend_profile, extend_profile_water_LR
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib import cm
from cauer import *
from scipy.integrate import quad, quadrature
%matplotlib qt
from matplotlib import cm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib as mpl

mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['figure.titlesize'] = 15
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20

from mfpt import *


def get_pbc_rate_matrix(F,D):
    n = len(F)
    rate = init_rate_matrix(n, F, D, pbc=True)
    return rate

def get_rate_matrix(F,D):
    n = len(F)
    rate = init_rate_matrix(n,F,D,pbc=True)
    # undo PBC
    rate[0,-1] = 0.
    rate[-1,0] = 0.
    # Constant A B BCs
    rate[0,0] = rate[0,1] = 0
    rate[-1,-1] = rate[-1,-2] = 0

    return rate

def propagate_with_R(time,R):
    prop = scipy.linalg.expm(time*R)
    init = np.zeros(len(prop))
    init[0] = 1.
    profile = np.dot(prop,init)
    return profile

def diagonalize_rate_matrix(R):
    eigval, eigvec = np.linalg.eig(R)
    # Sort from low to high, such that eigvec[:,-1] and eigvec[:,-2] are the lambda=0 eigvectors
    idx_sort = np.flip(eigval.argsort()[::-1])
    eigval = eigval[idx_sort]
    eigvec = eigvec[:,idx_sort]
    Diag = np.diag(eigval)
    Q = eigvec # R = Q.D.Q^-1, and thus exp(Rt) = Q.exp(Dt).Q^-1
    Qinv = np.linalg.inv(Q)
    max_diff = np.max(np.abs(np.matmul(np.matmul(Q,Diag),np.linalg.inv(Q))-R))
    print("||R - Q.D.Q^-1||_max = "+str(max_diff))
    return Diag,Q,Qinv

def propagate_with_diagonal(time,Diag,Q,Qinv):
    Diag_exp = np.zeros_like(Diag)
    for i in range(len(Diag)):
        Diag_exp[i,i] = np.exp(Diag[i,i]*time)
    prop = np.matmul(np.matmul(Q,Diag_exp),Qinv)
    init = np.zeros(len(prop))
    init[0]=1.
    profile = np.dot(prop,init)
    return profile

def get_eigenrates(R):
    eigval, eigvec = np.linalg.eig(R)
    # Sort from low to high, such that eigvec[:,-1] and eigvec[:,-2] are the lambda=0 eigvectors
    idx_sort = np.flip(eigval.argsort()[::-1])
    eigval = eigval[idx_sort]
    eigvec = eigvec[:,idx_sort]
    return eigval, eigvec

def get_taus(D,F,dz=1.):
    mid = int(len(D)/2)
    left = int(len(D)/4)
    right = len(D)-left
    F_L = F[0]
    F_R = F[-1]
    R_L = np.sum(1./(D[1:mid]*np.exp(-(F[1:mid]-F_L))))*dz
    R_R = np.sum(1./(D[mid:-1]*np.exp(-(F[mid:-1]-F_R))))*dz
    R = np.sum(1./(D*np.exp(-(F-F_L))))*dz
    R_Lmin = np.sum(1./(D[1:left]*np.exp(-(F[1:left]-F_L))))*dz
    R_Rmin = np.sum(1./(D[right:-1]*np.exp(-(F[right:-1]-F_R))))*dz
    C = np.sum(np.exp(-(F[1:-1]-F_L)))*dz
    CL = np.sum(np.exp(-(F[1:-1]-F_L)))*dz
    CR = np.sum(np.exp(-(F[1:-1]-F_R)))*dz
    CmidR = np.sum(np.exp(-(F[left:right]-F_R)))*dz
    CmidL = np.sum(np.exp(-(F[left:right]-F_L)))*dz
    R_L_avg = np.sum([np.exp(-(F[i]-F_L))*np.sum(1./(D[1:i]*np.exp(-(F[1:i]-F_L))))*dz for i in range(1,mid)])/np.sum(np.exp(-(F[1:mid]-F_L)))
    R_R_avg = np.sum([np.exp(-(F[i]-F_R))*np.sum(1./(D[i:-1]*np.exp(-(F[i:-1]-F_R))))*dz for i in range(mid,len(F)-1)])/np.sum(np.exp(-(F[mid:-1]-F_R)))

    
    tau_mid = C*R_L*R_R/(R_L+R_R)
    tau_split = C*R_Lmin*R_Rmin/(R_Lmin+R_Rmin)
    tau_RC = C*R/4
    tau_new = (1/CL/R_L + 1/CR/R_R)**(-1)
    tau_newsplit = (1/CL/R_Lmin + 1/CR/R_Rmin)**(-1)
    tau_newest = (1/CL/R_L_avg + 1/CR/R_R_avg)**(-1)
    
    return [tau_mid,tau_split,tau_RC,tau_new,tau_newsplit,tau_newest]

In [2]:
# Cutting the membrane part out of F and D profiles #
#####################################################

fn = "fit.popc.select.dat"
F,D,edges = read_F_D_edges(fn)
Drad = D # Not used, but just such that it has same dimensions.

nbins = len(F)
zpbc = edges[-1] - edges[0]
dz = (edges[-1]-edges[0])/(len(edges)-1.) #angstrom
dt = 1.

# BILAYER = bin [12,...,87] = 76 bins
# add 1 bin resembling water to the left and the right
# so 1 bin water + bilayer + 1 bin water = [11,...,88] = 78 bins
# And that is what we select, and define as 'short':
st = 11
end = 88

v_short = F[st:end+1]
d_short = D[st:end+1]
drad_short = Drad[st:end+1]
edges_short = edges[st:end+2] # has an extra element
z_array = np.array([i*dz for i in range(len(d_short))])

# Select the correct water bins
# Yes, this introduces a discontinuity in the F and D profiles,
# but it's a more realistic water profile.
v_short[0] = F[0]
v_short[-1] = F[-1]
d_short[0] = D[0]
d_short[-1] = D[-1]
# idx -1 and 0 have the same values, don't worry.


def get_properties(F, D, F_ref, D_ref, dz=1.):
    C = np.sum(np.exp(-(F-F_ref)))*dz 
    R = np.sum(1./(D*np.exp(-(F-F_ref))))*dz
    return C, R

print(get_properties(v_short[1:-1], d_short[1:-1], v_short[0], d_short[0], dz=dz))


(325.58738958637207, 385.42433560107196)


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


In [3]:
times = []
for time in np.arange(0.1,1,0.01):
    times.append(time)
for time in np.arange(1,10,0.1):
    times.append(time)
for time in np.arange(10,100,1):
    times.append(time)
for time in np.arange(100,1000,10):
    times.append(time)
for time in np.arange(1000,10000,100):
    times.append(time)
for time in np.arange(10000,100000,1000):
    times.append(time)
for time in np.arange(100000,1000000,10000):
    times.append(time)
# # For concentration profile evolutoin
# for time in np.arange(1000000,10000000,100000):
#     times.append(time)
# For storage plots
for time in np.arange(1000000,5000000,10000):
    times.append(time)

times = np.array(times)

In [4]:
# Let's plot the profiles we use 
fig, ax = plt.subplots(1,1,figsize=(8,6))
ax.plot(z_array, v_short, label="F")
ax.plot(z_array, d_short, label="D")
ax.set_xlabel("z (A)")
ax.set_ylabel("F (kT)")
ax.legend()
fig.tight_layout()
fig.show()

libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


In [5]:
# We create a five-stack, and charge the stack from the left.
mult = 5
vmulti, dmulti, dradmulti, edgesmulti =\
    extend_profile(v_short[1:-1], d_short[1:-1],
                   drad_short[1:-1], edges_short[1:-1],
                   mult)
extraF = [(v_short[-1]+v_short[0])*0.5 for i in range(1)]
extraD = [d_short[-1] for i in range(1)]
extraDrad = [drad_short[-1] for i in range(1)]
vmult = np.array(extraF + vmulti.tolist() + extraF)
dmult = np.array(extraD + dmulti.tolist() + extraD)
dradmult = np.array(extraDrad + dradmulti.tolist() + extraDrad)
R = get_rate_matrix(vmult, np.log(dmult/dz**2))
Diag,Q,Qinv = diagonalize_rate_matrix(R)
evals, evecs = get_eigenrates(R)
S_inf = np.sum(evecs[1:-1,-2]/evecs[0,-2])*dz
inf_prof = evecs[:,-2]/evecs[0,-2]
profiles = []
for i,time in enumerate(times):
    profile = propagate_with_diagonal(time,Diag,Q,Qinv)
    profiles.append(profile)
profiles = np.array(profiles)
np.save("onlywaterLR_POPConly_mult"+str(mult)+"_profiles.npy",np.array(profiles))
np.save("onlywaterLR_POPConly_mult"+str(mult)+"_times.npy",np.array(times))
np.save("onlywaterLR_POPConly_mult"+str(mult)+"_R.npy",R)

||R - Q.D.Q^-1||_max = 1.6239787292704477e-13


In [ ]:
tau = evals[-3]**(-1)*-1
print("tau = ",tau)

tau =  325896.23447586305


In [ ]:
# We plot the concentration profile propagation
mtimes = times
colmap = cm.get_cmap("tab20c_r")

t0 = mtimes[0]
t1 = mtimes[-1]

L = np.log(t1)-np.log(t0)
t_shift = -1. * np.log(t0)

colors = (np.log(mtimes)+t_shift)/L
print("colors range from: ",colors[0], " to ", colors[-1])

fig,ax=plt.subplots()
for i, prof in enumerate(profiles):
    if i == 0:
        pass
    elif i >= len(mtimes)-1:
        pass
    else:
        ax.plot([j*dz for j in range(len(prof))],prof,color=colmap(colors[i]))
ax.plot([j*dz for j in range(len(prof))],inf_prof,color="k",lw=2)

ax.set_xlabel(r"$z$ [$\mathrm{\AA}$]")
ax.set_ylabel(r"$c/c_L$", fontsize=25)

inaxes = inset_axes(ax,width="5%",height="70%",loc=1,bbox_to_anchor=(-0.0001,-0.10,1,1),bbox_transform=ax.transAxes,borderpad=0)
inaxes.imshow(np.vstack((colors,colors)).T, cmap=colmap, aspect = 0.04)
tenfold_ids = [90*i for i in range(int(len(times)/90+1))]
ticktimes = [0.1*10**i for i in range(len(tenfold_ids))]
inaxes.set_yticklabels(["{:.0e}".format(tick/1000) for tick in ticktimes], fontsize=15)
inaxes.set_yticks(tenfold_ids)
inaxes.set_xticks([])
#inaxes.set_title(r"time $t$ [ns]"+"\n",fontsize=20,loc='right')
# add a title box manually 
inaxes.text(-10, 0.95, r"time $t$ [ns]", fontsize=20, transform=inaxes.transAxes,
        verticalalignment='bottom', horizontalalignment='center')
fig.tight_layout()
fig.show()
#fig.savefig("propagation-5-stack.pdf")

colors range from:  0.0  to  1.0


/home/wouter/anaconda/envs/openmm/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator
/home/wouter/anaconda/envs/openmm/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.


In [6]:
# Now we create the ladder network for the five-stack.
aveD = np.array([(d_short[0]+d_short[-1])/2.] + ((d_short[1:]+d_short[:-1])/2.).tolist())
v_shortref = v_short[0]
part = np.exp(-(v_short-v_shortref))    # no units, partition function, not normalized
dRdx = 1./(part*aveD)         # permeation resistance per unit length, unit ps/A^2
Resis = np.sum(dRdx[1:-1])*dz  # resistance to permeability, unit ps/A
# integrate from x=-25.5=edges[st] to x=25.5=edges[end]
Perm = 1./Resis                      # permeability, unit A/ps
conc = np.exp(-v_short+v_short[0])
SfromF = np.sum(conc[1:-1])*dz
# We print the variables 
print("Resis = ",Resis)
print("Perm = ",Perm)
print("SfromF = ",SfromF)

Resis =  383.2219879609047
Perm =  0.0026094536102193003
SfromF =  325.58738958637207


In [7]:
# We solve the ladder network using the analytical cauer solution
C = SfromF
R = Resis
RCdiv2 = R*C/2  # This is the prefactor for analytical eigenvalues for cauer

# RC-ladder-lists
states_RC = []

eva_RC, P_RC = get_eigen_from_model(5, RC=RCdiv2)
D_RC = np.diag(eva_RC)
V_RC = get_sources()
B_RC = get_Bn(5, RC=RCdiv2)

# Get the voltages over the states
for t in times:
    states_RC.append(get_states_at_t(B_RC, P_RC, D_RC, V_RC, t))
states_RC = np.array(states_RC)

# Also getting the steady state solution for this 
# set-up (i.e. for these input sources)
ss_RC = get_steady_state(B_RC,P_RC,D_RC,V_RC)

In [8]:
# Now we will compare the analytical solution with the discretized Smoluchowski

# A) Smoluchwoski
# A1) we have to sum the concentrations over the membranes 
len_mem = len(v_short)
len_pmem = len_mem -2
M = 5  # number of membranes in bilayer
memS = [[] for i in range(M)]
for prof in profiles:
    for i in range(M):
        memS[i].append(np.sum(prof[1+len_pmem*i:1+len_pmem*(i+1)]*dz))
memS = np.array(memS)
# A2) We get the steady state values
R = np.load("onlywaterLR_POPConly_mult"+str(M)+"_R.npy")
evals, evecs = get_eigenrates(R)
Sinf_mems = [np.sum(evecs[1+len_pmem*i:1+len_pmem*(i+1), -2]/evecs[0,-2]*dz)
             for i in range(M)]

# B) Cauer ladder network
# B1) membrane storages are found by multioplying the voltage with the capacitance
storages_RC = states_RC * C
# B2) steady state values
ss_storage_RC = ss_RC * C 

# We now plot the results
fig,ax=plt.subplots()
ax.plot(times/1000000, memS[0]/Sinf_mems[0], '-o', c="k", ms=6, label="ref")
for i in range(M):
    ax.plot(times/1000000, memS[i]/Sinf_mems[i], '-o', c="k", ms=6)
    ax.plot(times/1000000, storages_RC[:,i]/ss_storage_RC[i], '-x', ms=5, label=r"$i=$"+str(i+1))
# make a legend title
ax.legend(fontsize=20) #, title=r"$\frac{S_i(t)}{S_i(t\rightarrow{}\infty)}$", title_fontsize=30)
ax.set_xlim(0, 2.1)
ax.set_xlabel(r"time $t$ [$\mu$s]")
ax.set_ylabel(r"$\dfrac{S_i(t)}{S_i(t\rightarrow{}\infty)}$", fontsize=25)
fig.tight_layout()
fig.show()
#fig.savefig("storage-5-stack.pdf")


In [10]:
np.save("memS_5stack.npy", memS)
np.save("times.npy", times)

In [150]:
# Now we will look at the storage and time-constant vs myelin stack size M

In [200]:
largest_timeconstants = []
equilibrium_storages = []
multlist = np.arange(1,201)
for M in multlist:
    eva_RC, P_RC = get_eigen_from_model(M, RC=RCdiv2)
    D_RC = np.diag(eva_RC)
    V_RC = get_sources(Vl = 1., Vr = 1.)
    B_RC = get_Bn(M, RC=RCdiv2)
    ss_RC = get_steady_state(B_RC,P_RC,D_RC,V_RC)
    largest_timeconstants.append(eva_RC[-1]**(-1)*(-1))
    equilibrium_storages.append(np.sum([el*C for el in ss_RC]))

# We also get an analytical solution to the largest timevalue...
analytical_timeconstants = [-1.*RCdiv2/(np.cos(np.pi/k)-1) for k in multlist]

largest_timeconstants = np.array(largest_timeconstants)
equilibrium_storages = np.array(equilibrium_storages)
analytical_timeconstants = np.array(analytical_timeconstants)

In [234]:
fig, ax = plt.subplots()
ax.plot(multlist, analytical_timeconstants*10**(-6), '-', c="r", lw=8,
        label = r"$RC\cdot\left(2-2\cos{\left(\frac{\pi}{M}\right)}\right)^{-1}}$")
ax.plot(multlist, largest_timeconstants*10**(-6), 'x', c="k", ms=5,
        label = r"$\tau_{\mathrm{max}}(M)$")
ax.set_xlabel(r"number of membranes $M$")
ax.set_ylabel(r"$\tau_{\mathrm{max}}$ [$\mu$s]")
ax.legend(fontsize=20)
fig.tight_layout()
fig.show()
fig.savefig("largest-timeconstant-vs-M.pdf")

In [233]:
fig, ax = plt.subplots()
ax.plot(multlist, [k*C for k in multlist], '-', c="r", lw=8,
        label = r"$M\cdot C$")
ax.plot(multlist, equilibrium_storages, 'x', c="k", ms=5,
        label = r"$S(M)$")
ax.set_xlabel(r"number of membranes $M$")
ax.set_ylabel(r"$S$ [mol/m$^2$]")
ax.legend(fontsize=20)
# scientific notation in y-axis
ax.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
fig.tight_layout()
fig.show()
fig.savefig("equilibrium-storage-vs-M.pdf")